## TSNE stuff

In [1]:
# coding='utf-8'
"""t-SNE对手写数字进行可视化"""
from time import time
import numpy as np
import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.manifold import TSNE

## MSVC stuff

In [2]:
import sys
sys.path.append("..")
import numpy as np
from lib.dataset import makeDataLoader
from lib.models import makeModel
from lib.loss import makeLoss
from lib.utils import evaluate
import torch

In [3]:
device = torch.device("cuda:0")
# data
n_samples, classes, testloader = makeDataLoader(train=False, bw=[32,16,8], dataset_name="modelnet10")
# loss
criterion = makeLoss("margin_loss", nclasses=len(classes))
# model
_, model = makeModel("msvc",
        "../logs/msvc_modelnet10_marginloss_norotate",
        nclasses=len(classes),
        device=device,
        is_distributed=True,
        use_residual_block=True,
        continue_training=True)
_

52

In [ ]:
result_eval = evaluate(model, criterion, testloader, device, nclasses=len(classes), plot=True)

In [ ]:
result_eval

In [ ]:
def get_embeddings(model, testloader):
    model.eval()
    embeddings = []
    labels = []
    for batch_idx, (data, target) in enumerate(testloader):
        if isinstance(data, list):
            inputs = [i.to(device) for i in data]
            data = inputs
        else:
            data = data.to(device)
        prediction = model(data)
        embeddings.append(prediction.cpu().detach().numpy())
        labels.append(target.cpu().detach().numpy())
    embeddings = np.concatenate(embeddings, axis=0)
    labels = np.concatenate(labels, axis=0)
    n_samples, n_features = embeddings.shape
    return embeddings, labels, n_samples, n_features

def plot_embedding(data, label, title):
    x_min, x_max = np.min(data, 0), np.max(data, 0)
    data = (data - x_min) / (x_max - x_min)

    fig = plt.figure()
    ax = plt.subplot(111)
    for i in range(data.shape[0]):
        plt.text(data[i, 0], data[i, 1], str(label[i]),
                 color=plt.cm.Set1(label[i] / 10.),
                 fontdict={'weight': 'bold', 'size': 9})
    plt.xticks([])
    plt.yticks([])
    plt.title(title)
    plt.figure(dpi=300,figsize=(24,16))
    return fig


def main(func=None):
    if not func:
        return
    data, label, n_samples, n_features = func(model, testloader)
    print('Computing t-SNE embedding')
    tsne = TSNE(n_components=2, init='pca', random_state=0)
    t0 = time()
    result = tsne.fit_transform(data)
    fig = plot_embedding(result, label,
                         't-SNE embedding of ModelNet10'
                         % (time() - t0))
    plt.show(fig)


main(get_embeddings)

In [ ]:
data, label, n_samples, n_features = get_embeddings(model, testloader)

In [ ]:
def get_data(model, testloader):
    embeddings = []
    labels = []
    for batch_idx, (data, target) in enumerate(testloader):
        e=np.concatenate([data[i].reshape(4,-1).cpu().detach().numpy() for i in range(3)], axis=1)
        embeddings.append(e)
        labels.append(target)
    embeddings = np.concatenate(embeddings, axis=0)
    labels = np.concatenate(labels, axis=0)
    n_samples, n_features = embeddings.shape
    return embeddings, labels, n_samples, n_features
embeddings, labels, n_samples, n_features = get_data(model, testloader)

In [ ]:
main(get_data)